In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

## Loading data
First we have to load data from the various tab-separated files

Source: https://archive.ics.uci.edu/ml/datasets/diabetes

In [2]:
path = 'data/raw/'
df = pd.DataFrame()
df_list=[]
#df_list = [pd.read_csv(path + file ,sep='\t',header=None) for file in os.listdir(path)]
for i,file in enumerate(os.listdir(path)):
    df_temp = pd.read_csv(path + file ,sep='\t',header=None)
    df_temp['id_patient'] = i
    df = df.append(df_temp,ignore_index=True)

/Users/pkaehler/work/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
df.columns = ['date', 'time', 'code', 'value','id_patient']

## Data description
The data set contains a diabetes patient record
Columns:
1. date
2. time
3. code describes (#TODO)
4. value (#TODO) 

After a short Google "Research", we define `value` as too high if above 130 (before lunch) and 180 (if within 2 hours after lunch)

Codes can be lookuped in a second dataframe (df_codes)

In [4]:
df.head()

,date,time,code,value,id_patient
0,05-20-1991,08:00,58.0,101,0
1,05-20-1991,08:00,33.0,5,0
2,05-20-1991,08:00,34.0,27,0
3,05-20-1991,12:00,60.0,89,0
4,05-20-1991,12:00,33.0,3,0


We have null values in every column. We should check if it reasonable low, before just removing them

In [5]:
for column in df.columns:
    print(f'nulls in {column}: {len(df[df[column].isnull()])}')

nulls in date: 34
nulls in time: 1
nulls in code: 1
nulls in value: 34
nulls in id_patient: 0


We have to think about handling null-Values.
*remove* or *replace* (eg with the mean() or a constant)

In [7]:
before = len(df)
df_clean = df.dropna()
# I was just confused by the float-type of the code
df_clean['code'] = df_clean['code'].astype('int64')
# and some date transformation, so that we can use functions
df_clean['timestamp'] = pd.to_datetime(df_clean['date'].astype(str) + ' ' + df_clean['time'],format='%m-%d-%Y %H:%M',errors='coerce')
after = len(df_clean)
print(f'removed {before-after} entries, length dataframe without null: {after}')

removed 67 entries, length dataframe without null: 29264


/Users/pkaehler/work/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/pkaehler/work/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


We still have unclean data in the target column. Next step is to check them out and if necessary, remove them.

In [8]:
df_clean.head()

,date,time,code,value,id_patient,timestamp
0,05-20-1991,08:00,58,101,0,1991-05-20 08:00:00
1,05-20-1991,08:00,33,5,0,1991-05-20 08:00:00
2,05-20-1991,08:00,34,27,0,1991-05-20 08:00:00
3,05-20-1991,12:00,60,89,0,1991-05-20 12:00:00
4,05-20-1991,12:00,33,3,0,1991-05-20 12:00:00


In [9]:
df_clean.loc[pd.to_numeric(df_clean['value'], errors='coerce').notnull()==False]

,date,time,code,value,id_patient,timestamp
12256,11-06-1989,11:00,57,0'',29,1989-11-06 11:00:00
19595,10-31-1989,12:00,60,0Hi,51,1989-10-31 12:00:00
19629,11-04-1989,18:00,62,0Hi,51,1989-11-04 18:00:00
19635,11-05-1989,12:00,60,0Hi,51,1989-11-05 12:00:00
19643,11-06-1989,12:00,60,0Hi,51,1989-11-06 12:00:00
19647,11-06-1989,22:00,48,0Hi,51,1989-11-06 22:00:00
19661,11-08-1989,18:00,62,0Lo,51,1989-11-08 18:00:00
20037,12-25-1989,22:00,48,0Hi,51,1989-12-25 22:00:00


-> we have to check what 0Hi and 0Lo means
For now, drop it

In [10]:
before = len(df_clean)
df_clean.drop(df_clean.loc[pd.to_numeric(df_clean['value'], errors='coerce').notnull()==False].index,inplace=True)
after = len(df_clean)
print(f'removed {before-after} entries, length dataframe without null: {after}')

removed 8 entries, length dataframe without null: 29256


/Users/pkaehler/work/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


### descriptive analysis, cleansing and outliers

#### General statistics and correlations

In [ ]:
print(f'Amount data points: {len(df)}')

#### Date statistics

In [ ]:
amount_dates = df['date'].nunique()
print(f'Amount days (uncleaned): {amount_dates}')

In [ ]:
df.groupby('date').size().hist(bins=20)

In [ ]:
min_per_day = df.groupby('date').size().min()
max_per_day = df.groupby('date').size().max()
print(f'Min. data points per day: {min_per_day}')
print(f'Min. data points per day: {max_per_day}')
print(df.groupby('date').size().describe(percentiles=[.75,.80,.95,.99]))

### data set description
1. The amount of data points per day is right-skewed -> 75% of days have less than 31, the remaining days hold between 32 and max 126 points
2. 

#### Time statistics

In [ ]:
amount_times = df['time'].nunique()
print(f'Amount unique points in time (uncleaned): {amount_times}')

#### Code statistics

In [ ]:
amount_codes = df['code'].nunique()
print(f'Amount unique points in code (uncleaned): {amount_codes}')

#### Value statistics

In [ ]:
amount_values = df['value'].nunique()
print(f'Amount unique points in value (uncleaned): {amount_values}')

# Classification


## Transformation and feature engineering

When Should I Use Log Transformation?
Many possible transformations exist. However, you should only use a log transformation if:

Your data is highly skewed to the right (i.e. in the positive direction).
(https://www.statisticshowto.datasciencecentral.com/probability-and-statistics/skewed-distribution/)

In [ ]:
from sklearn.metrics import precision_recall_curve
y_train = [1]
y_scores = [1]
precisions, recalls, thresholds = precision_recall_curve(y_train, y_scores)

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1])

In [ ]:
plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

# Testing area

In [ ]:
len(df_clean[df_clean.duplicated()])
#df_clean.duplicated()

In [ ]:
# TODO: when is a duplicate a duplicate?
df_clean[df_clean.duplicated()]

In [ ]:
df_clean[df_clean['id_patient']==12]